# Simple Neuron Network
In this section, we build very simply Nnabla model and save the model in a NNB file.

In [1]:
import nnabla as nn
import nnabla.functions as F
import nnabla.parametric_functions as PF
import numpy as np
import nnabla.initializer as I
from save_nnp import save_nnp

nn.parameter.clear_parameters()
rng = np.random.seed(31024)
initializer = I.UniformInitializer((-0.1, 0.1), rng=rng)

2020-05-05 15:04:59,928 [nnabla][INFO]: Initializing CPU extension...


In [2]:
batch_size = 1
nnp_file_name = 'simple_nn_0.nnp'
nnb_file_name = 'simple_nn_0.nnb'

### Build model

In [3]:
x = nn.Variable((1, 5))
y = PF.affine(x, 1, w_init=initializer)

In [4]:
x.d = np.array([1,2,3,4,5])
print(x.d)

[[1 2 3 4 5]]


In [5]:
y.forward()
y.d

array([[0.7102661]], dtype=float32)

### save model

In [6]:
content = save_nnp(nnp_file_name, input={'x': x}, output={'y': y}, batchsize=batch_size)
print(content)

2020-05-05 15:05:01,670 [nnabla][INFO]: Saving simple_nn_0.nnp as nnp
2020-05-05 15:05:01,672 [nnabla][INFO]: Saving /tmp/tmpx9ph3yml/network.nntxt as prototxt
2020-05-05 15:05:01,675 [nnabla][INFO]: Parameter save (.protobuf): /tmp/tmpx9ph3yml/parameter.protobuf


{'networks': [{'name': 'NN-0', 'batch_size': 1, 'outputs': {'y': <Variable((1, 1), need_grad=True) at 0x7f9fcb42a470>}, 'names': {'x': <Variable((1, 5), need_grad=False) at 0x7f9fcb405fb0>}}], 'executors': [{'name': 'Runtime', 'network': 'NN-0', 'data': ['x'], 'output': ['y']}]}


#### Convert from NNP to NNB

In [ ]:
# Convert from NNP to NNB
import subprocess, sys, os
out = subprocess.Popen(['nnabla_cli', 'convert', '-b', '%d' % batch_size, nnp_file_name, nnb_file_name])
assert(os.path.exists(nnb_file_name))
assert(os.path.isfile(nnb_file_name))

#### Convert from binary NNB to C-style NNB

https://github.com/Jamesits/bin2array

### load model

In [ ]:
from nnabla.utils.nnp_graph import NnpLoader
# Read a .nnp file.
nnp = NnpLoader(nnp_file_name)

In [ ]:
nw_name = nnp.get_network_names()[0]
net = nnp.get_network(nw_name)
print(net.variables)

In [ ]:
print(net.inputs)
print(net.outputs)
x = net.inputs['x']
y = net.outputs['y']
print(x)
print(y)

In [ ]:
x.d = np.array([11,12,13,14,15])
y.forward()  # You can execute a sub graph.
print("y: %s" % y.d)

# Simple CNN

In [ ]:
import nnabla as nn
import nnabla.functions as F
import nnabla.parametric_functions as PF
import numpy as np
import nnabla.initializer as I

nn.parameter.clear_parameters()

rng = np.random.seed(31024)
initializer = I.UniformInitializer((-0.1, 0.1), rng=rng)

# create 16 random images 256x256 (RGB)
x = nn.Variable((16, 3, 256, 256))
x.d = np.random.random(x.shape)  # random input, just for example.

y0 = PF.convolution(x, outmaps=64, kernel=(3, 3), pad=(1, 1), stride=(2, 2), w_init=initializer, name="conv1", with_bias=False)
y1 = F.relu(y0)
y2 = PF.convolution(y1, outmaps=128, kernel=(3, 3), pad=(1, 1), stride=(2, 2), w_init=initializer, name="conv2", with_bias=False)
y3 = F.relu(y2)
y4 = F.average_pooling(y3, kernel=y3.shape[2:])
y5 = PF.affine(y4, 1, w_init=initializer)
loss = F.mean(F.abs(y5 - 1.))
loss.forward()  # Execute forward

# We can check the current gradient of parameter.
print("******* forward *******")
print(nn.get_parameters()["conv1/conv/W"].g)

loss.backward()
print("******* backward *******")
print(nn.get_parameters()["conv1/conv/W"].g)

## CNN with static graph
In this section, we demonstrate:
 - building a simple graph
 - training model
 - running prediction (with the trained model)
 - saving trained model
 - loading trained model
 - running prediction (with the loaded model)

In [ ]:
import sys
import numpy as np
from sklearn.datasets import load_digits  # Only for dataset

import nnabla as nn
import nnabla.functions as F
import nnabla.parametric_functions as PF
import nnabla.solvers as S
import nnabla.initializer as I
from nnabla.monitor import tile_images
from nnabla.utils.data_iterator import data_iterator_simple

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

nn.clear_parameters()
nn.reset_array_preference()

# set graph
CNN_GRAPH = 1

batch_size = 64
h5_file_name = 'static_graph.h5'
nnp_file_name = 'cnn_static_graph.nnp'
nnb_file_name = 'cnn_static_graph.nnb'
csrc_dir_name = 'cnn_static_graph_csrc'

rng = np.random.seed(31024)
initializer = I.UniformInitializer((-0.1, 0.1), rng=rng)

imshow_opt = dict(cmap='gray', interpolation='nearest')

In [ ]:
imshow_opt = dict(cmap='gray', interpolation='nearest')

def plot_stats(digits):
    print("Num images:", digits.images.shape[0])
    print("Image shape:", digits.images.shape[1:])
    print("Labels:", digits.target[:10])
    plt.imshow(tile_images(digits.images[:64, None]), **imshow_opt)


def data_iterator_tiny_digits(digits, batch_size=batch_size, shuffle=False, rng=None):
    def load_func(index):
        """Loading an image and its label"""
        img = digits.images[index]
        label = digits.target[index]
        return img[None], np.array([label]).astype(np.int32)
    return data_iterator_simple(load_func, digits.target.shape[0], batch_size, shuffle, rng, with_file_cache=False)

In [ ]:
# get dataset
digits = load_digits(n_class=10)
plot_stats(digits)
print('type(digits) = %s' % type(digits))

In [ ]:
data = data_iterator_tiny_digits(digits, batch_size=batch_size, shuffle=True)
img, label = data.next()
plt.imshow(tile_images(img), **imshow_opt)
print("labels:", label.reshape(8, 8))
print("Label shape:", label.shape)
print('type(img) = %s, img.shape = %s' % (type(img), img.shape))
print('type(label) = %s' % type(label))

print(img[0])

### build a graph
> Note: we have to run only one section:
    - build simple graph
    - build more complex graph

#### build simple graph

In [ ]:
if CNN_GRAPH == 0:
    # Forward pass
    x = nn.Variable(img.shape)  # Define an image variable
    with nn.parameter_scope("affine1"):
        y = PF.affine(x, 10)  # Output is 10 class

    # Building a loss graph
    t = nn.Variable(label.shape)  # Define an target variable
    loss = F.mean(F.softmax_cross_entropy(y, t))  # Softmax Xentropy fits multi-class classification problems

    print("Printing shapes of variables")
    print('x:', x.shape)
    print('y:', y.shape)
    print('t:', t.shape)
    print('loss:', loss.shape)  # empty tuple means scalar

#### build more complex graph

In [ ]:
if CNN_GRAPH == 1:
    # Forward pass
    x = nn.Variable(img.shape)  # Define an image variable
    with nn.parameter_scope("affine1"):
        y0 = PF.convolution(x, outmaps=batch_size, kernel=(3, 3), pad=(1, 1), stride=(2, 2), w_init=initializer, name="conv1", with_bias=False)
        y1 = F.relu(y0)
        # since the image is already too small, pooling does not make sense
        # y2 = F.max_pooling(y1, kernel=y1.shape[2:])
        y = PF.affine(y1, 10)  # Output is 10 class

    # Building a loss graph
    t = nn.Variable(label.shape)  # Define an target variable
    loss = F.mean(F.softmax_cross_entropy(y, t))  # Softmax Xentropy fits multi-class classification problems

    print("Printing shapes of variables")
    print('x:', x.shape)
    print('y0:', y0.shape)
    print('y1:', y1.shape)
    print('y:', y.shape)
    print('t:', t.shape)
    print('loss:', loss.shape)  # empty tuple means scalar

### Stochastic Gradient Solver

In [ ]:
# Create a solver (gradient-based optimizer)
learning_rate = 1e-3
solver = S.Sgd(learning_rate)
solver.set_parameters(nn.get_parameters())  # Set parameter variables to be updated.

### training

In [ ]:
# Training
n_epochs = 1000
total_epochs = 0

In [ ]:
for _ in range(n_epochs):
    x.d, t.d = data.next()
    loss.forward()
    solver.zero_grad()  # Initialize gradients of all parameters to zero.
    loss.backward()
    solver.weight_decay(1e-5)  # Applying weight decay as an regularization
    solver.update()
    total_epochs = total_epochs + 1
    if total_epochs % 100 == 0:  # Print for each 10 iterations
        print('epoch=%d loss=%f' % (total_epochs, loss.d))

In [ ]:
# prediction
x.d, t.d = data.next()  # Here we predict images from training set although it's useless. 
y.forward()  # You can execute a sub graph.
plt.imshow(tile_images(x.d), **imshow_opt)
print("prediction:")
print(y.d.argmax(axis=1).reshape(8, 8))  # Taking a class index based on prediction score.

### save model

In [ ]:
def save_nnp(input, output, batchsize):
    runtime_contents = {
        'networks': [
            {'name': 'Validation',
             'batch_size': batchsize,
             'outputs': output,
             'names': input}],
        'executors': [
            {'name': 'Runtime',
             'network': 'Validation',
             'data': [k for k, _ in input.items()],
             'output':[k for k, _ in output.items()]}]}
    return runtime_contents

In [ ]:
# save parameters
nn.save_parameters(h5_file_name)

In [ ]:
img = nn.Variable(x.shape)
lab = nn.Variable(t.shape)
img.d, lab.d = data.next()
if CNN_GRAPH == 0:
    content = save_nnp(input={'x': x}, output={'y': y, 'loss': loss, 't': t}, batchsize=batch_size)
elif CNN_GRAPH == 1:
    # content = save_nnp(input={'x': x}, output={'y': y, 'y0': y0, 'y1': y1, 'loss': loss, 't': t}, batchsize=batch_size)
    content = save_nnp(input={'x': x}, output={'y': y, 'y0': y0, 'y1': y1}, batchsize=batch_size)
print(content)

In [ ]:
import nnabla.utils.save as save
save.save(nnp_file_name, content)

### load model

In [ ]:
from nnabla.utils.nnp_graph import NnpLoader

# load h5
nn.load_parameters(h5_file_name)
# Read a .nnp file.
nnp = NnpLoader(nnp_file_name)

In [ ]:
nw_name = nnp.get_network_names()[0]
net = nnp.get_network(nw_name)

In [ ]:
net.variables

In [ ]:
nw_name = nnp.get_network_names()[0]
print('nw_name = %s' % nw_name)
net = nnp.get_network(nw_name)
print(net)
# load h5
nn.load_parameters(h5_file_name)

In [ ]:
print(net.inputs)
print(net.outputs)

In [ ]:
x = net.inputs['x']
y = net.outputs['y']
print(x)
print(y)

In [ ]:
# prediction
print('data.position=%d' % data.position)
x.d, _ = data.next()  # Here we predict images from training set although it's useless. 
y.forward()  # You can execute a sub graph.
plt.imshow(tile_images(x.d), **imshow_opt)
print("prediction:")
print(y.d.argmax(axis=1).reshape(8, 8))  # Taking a class index based on prediction score.

### File format conversion

#### Convert from NNP to NNB

In [ ]:
# Convert from NNP to NNB
import subprocess, sys, os
out = subprocess.Popen(['nnabla_cli', 'convert', '-b', '%d' % batch_size, nnp_file_name, nnb_file_name])
assert(os.path.exists(nnb_file_name))
assert(os.path.isfile(nnb_file_name))

#### Convert from binary NNB to C-style NNB

https://github.com/Jamesits/bin2array

#### Convert from NNP to C source
This generated C source files are in the directory `csrc_dir_name`, with a GNUmakefile. This should be a good example of using the generated C source model.

Requirements before compiling the C source:
1. clone the official Nnabla C Runtime from https://github.com/sony/nnabla-c-runtime (without -fPIE flag) or https://github.com/mu-triv/nnabla-c-runtime (with -fPIE)
2. compile Nnabla C Runtime (`make`)
3. set these environment variables:
    * `LD_LIBRARY_PATH` and `LIBRARY_PATH` (example: `nnabla-c-runtime/build/_CPack_Packages/Linux/ZIP/nnabla-c-runtime-1.2.0.dev1_c1-Linux/lib`)
    * `C_INCLUDE_PATH` and `CPLUS_INCLUDE_PATH` (example: `nnabla-c-runtime/include`)
    * Or you will need to modify the GNUmakefile

The generated C source is compiled with the following command:
```
cd csrc_dir_name
make
```

In [ ]:
# Convert from NNP to CSRC
import subprocess
import pathlib
pathlib.Path(csrc_dir_name).mkdir(parents=True, exist_ok=True) 

out = subprocess.Popen(['nnabla_cli', 'convert', '-b', '%d' % batch_size, '-O', 'CSRC', nnp_file_name, csrc_dir_name])
assert(os.path.exists(nnb_file_name))
assert(os.path.isdir(csrc_dir_name))

#### save 64 raw images
These images are used with prediction later

In [ ]:
prediction_file = 'img_64010808.raw'

In [ ]:
with open(prediction_file, 'wb') as fh:
    print('data.position=%d' % data.position)
    x.d, _ = data.next()
    print('x.d.shape = %s' % str(x.d.shape))
    print('type = %s' % type(x.d[0,0,0,0]))
    fh.write(np.float32(x.d))
plt.imshow(tile_images(x.d), **imshow_opt)

#### Convert from binary raw images to C-style images

https://github.com/Jamesits/bin2array

### display the prediction results
> Note: execute this cell after the CSRC inference execution

This cell displays the prediction result and allows to compare the result with the reference images.

I assume that we execute the CSCR inference this way

`./Validation_example ../img_64010808.raw output`

> `img_64010808.raw` is defined in `prediction_file`
>
> `output` is the header of the output file names

After executing the prediction in CSRC, we obtain several output data, such as `output_0.bin`, `output_1.bin`, `output_2.bin`

In [ ]:
import os
yinf = nn.Variable(y.shape)
prediction_result_file = os.path.join(csrc_dir_name, 'output_0.bin')
with open(prediction_result_file, 'rb') as fh:
    fbytes = fh.read()

afloats = np.frombuffer(fbytes, dtype=np.float32)
yinf.d = np.reshape(afloats, yinf.shape)
print(yinf.d.argmax(axis=1).reshape(8, 8))  # Taking a class index based on prediction score.

# load the reference images
yref = nn.Variable(x.shape)
with open(prediction_file, 'rb') as fh:
    yref_d = fh.read()
yref.d = np.reshape(np.frombuffer(yref_d, dtype=np.float32), yref.shape)
# display the reference images
plt.imshow(tile_images(yref.d), **imshow_opt)

# Annex

## Functions

In [ ]:
import numpy as np
x=3; y=4; z=5
a = np.random.randint(0, 10, size=(x,y,z))
b = np.random.randint(0, 10, size=(x,y,z))

### compare constants

In [ ]:
import nnabla as nn

nn.set_auto_forward(True)

x0 = nn.NdArray((a.shape))
x0.data = a
x1 = nn.NdArray((b.shape))
x1.data = b
print("x0=", x0.data)
print("x1=", x1.data)

In [ ]:
r = nn.functions.equal(x0, x1, n_outputs=-1, outputs=None)
print(r, r.data)

In [ ]:
r = nn.functions.greater(x0, x1, n_outputs=-1, outputs=None)
print(r, r.data)

### compare variables

In [ ]:
x2 = nn.Variable(x0.shape)
x2.data = x0
x3 = nn.Variable(x1.shape)
x3.data = x1

In [ ]:
r = nn.functions.greater(x2, x3, n_outputs=-1, outputs=None)
print(r, r.d)

## minimum

In [ ]:
import nnabla.functions as F

x = nn.Variable.from_numpy_array(np.random.rand(2, 3, 4))
print(x)
print(x.d)

minval = F.min(x, axis=1)
print(minval)
print(minval.d)

## function relu

In [ ]:
import nnabla.parametric_functions as PF
import nnabla.functions as F

x = nn.Variable((2, 3, 8, 8))
x.d = np.random.random(x.shape)  # random input, just for example.

nn.parameter.clear_parameters()
with nn.parameter_scope('conv1') as param:
    conv_out = PF.convolution(x, 2, (5, 5))
    relu_out = F.relu(conv_out)
    
print(relu_out)
print(relu_out.d)

In [ ]:
import numpy as np
import nnabla as nn
import nnabla.parametric_functions as PF
import nnabla.functions as F
numbers = np.arange(start=-10.0, stop=10.0, step=.1, dtype=np.float)
print("numbers.shape=", numbers.shape)
x = nn.Variable(numbers.shape)
x.d = numbers
relu_out = F.relu(x)
print("x.shape=", x.shape)
print("relu_out.shape=", relu_out.shape)

from matplotlib import pyplot as plt
plt.plot(x.d, relu_out.d)

## sigmoid

In [ ]:

import nnabla.parametric_functions as PF
import nnabla.functions as F

x = nn.Variable((2, 3, 8, 8))
x.d = np.random.random(x.shape)  # random input, just for example.

nn.parameter.clear_parameters()
with nn.parameter_scope('conv2') as param:
    conv_out = PF.convolution(x, 2, (5, 5))
    sig_out = F.sigmoid(conv_out)
    
print(sig_out)
print(sig_out.d)

## Parametric functions

## Affine

In [ ]:
import numpy as np
import nnabla as nn
import nnabla.parametric_functions as PF

x = nn.Variable.from_numpy_array(np.array([[1, 2], [3, 4]]))
y = PF.affine(x, 4, name="y")
print("x")
print(x.d)
print("y")
print(y.d)